In [1]:
import pandas as pd

In [5]:
dfA = pd.read_csv("dsA.csv")
dfA.head()

,id,name,country,revenue_2022,profit,assets,market_value
0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b
1,1,icbc,china,doll_208.13 b,NaN,NaN,doll_214.43 b
2,"2,saudi arabian oil company (saudi aramco),sau...",NaN,NaN,NaN,NaN,NaN,NaN
3,3,jpmorgan chase,united states,doll_124.54 b,NaN,NaN,doll_374.45 b
4,4,china construction bank,china,doll_202.07 b,NaN,NaN,doll_181.32 b


In [17]:
df = pd.read_csv("predictions.csv")
df.head()

,match_score,ltable_id,rtable_id,ltable_name,ltable_country,ltable_revenue_2022,ltable_profit,ltable_assets,ltable_market_value,rtable_name,rtable_country,rtable_revenue_2022,rtable_profit,rtable_assets,rtable_market_value
0,0.784563,0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b,berkshire hathaway,united states,doll_276.09 b,doll_89.8 b,doll_958.78 b,doll_741.48 b
1,0.792979,9,9,toyota motor,japan,doll_281.75 b,NaN,NaN,doll_237.73 b,toyota motor,japan,doll_281.75 b,doll_28.15 b,doll_552.46 b,doll_237.73 b
2,0.781058,13,13,samsung group,south korea,doll_244.16 b,NaN,NaN,doll_367.26 b,samsung group,south korea,doll_244.16 b,doll_34.27 b,doll_358.88 b,doll_367.26 b
3,0.784673,18,18,verizon communications,united states,doll_134.35 b,NaN,NaN,doll_218.02 b,verizon communications,united states,doll_134.35 b,doll_21.52 b,doll_366.6 b,doll_218.02 b
4,0.786995,21,21,unitedhealth group,united states,doll_297.55 b,NaN,NaN,doll_490.15 b,unitedhealth group,united states,doll_297.55 b,doll_17.45 b,doll_221.24 b,doll_490.15 b


In [23]:
dict_match = {}
df = df.reset_index()
for index, row in df.iterrows():
    if row["match_score"] > 0.5:
        l_id = str(row['ltable_id'])
        r_id = str(row['rtable_id'])
        if l_id not in dict_match.keys():
            dict_match[l_id] = set()
        dict_match[l_id].add(r_id)
        if r_id not in dict_match.keys():
            dict_match[r_id] = set()
        dict_match[r_id].add(l_id)

In [42]:
def dict_list_to_dict_first_value(d):
    return {k: v[0] for k, v in d.items()}

In [43]:
dict_list_to_dict_first_value(dfA.loc[dfA['id'] == "0"].to_dict(orient="list"))

{'id': '0',
 'name': 'berkshire hathaway',
 'country': 'united states',
 'revenue_2022': 'doll_276.09 b',
 'profit': nan,
 'assets': nan,
 'market_value': 'doll_741.48 b'}

In [ ]:
for key, values in dict_match.items():
    

In [24]:
dict_match

{'0': {'0'},
 '9': {'9'},
 '13': {'13'},
 '18': {'18'},
 '21': {'21'},
 '24': {'24'},
 '27': {'27'},
 '30': {'30'},
 '32': {'32'},
 '33': {'33'},
 '40': {'40'},
 '41': {'41'},
 '47': {'47'},
 '52': {'52'},
 '53': {'53'},
 '58': {'58'},
 '59': {'59'},
 '60': {'60'},
 '62': {'62'},
 '63': {'63'},
 '65': {'65'},
 '68': {'68'},
 '70': {'207', '211', '476', '532', '669', '70'},
 '207': {'207', '476', '669', '70'},
 '211': {'211', '476', '532', '70'},
 '476': {'207', '211', '476', '532', '669', '70'},
 '532': {'211', '476', '532', '70'},
 '669': {'207', '476', '669', '70'},
 '71': {'71'},
 '72': {'72'},
 '74': {'74'},
 '75': {'75'},
 '76': {'76'},
 '16': {'1269', '79', '835'},
 '79': {'1269', '16', '79', '835'},
 '835': {'1269', '16', '79', '835'},
 '81': {'81'},
 '82': {'1137', '738', '82'},
 '738': {'1137', '738', '82'},
 '84': {'84'},
 '86': {'86'},
 '87': {'87'},
 '7': {'105',
  '1055',
  '116',
  '1189',
  '1195',
  '1244',
  '1245',
  '1444',
  '1492',
  '1493',
  '1524',
  '1526',
  '